I used a conda env with python 3.86

I installed spacy like this
conda install -c conda-forge spacy
conda install -c conda-forge spacy-lookups-data
python -m spacy download en_core_web_sm

In [5]:
import pandas as pd
import numpy as np
import json
import spacy

In [6]:
!ls

EDA.ipynb                      non_adverse_media_training.csv
adverse_media_training.csv     prep


In [45]:
am = pd.read_csv('adverse_media_training.csv.zip')
nam = pd.read_csv('non_adverse_media_training.csv.zip')

In [46]:
am.label.unique()

array([nan, 'am', 'delete', 'delete        ', 'random', 'nam', 'doubt',
       'neither', 'am, doubt', 'am '], dtype=object)

In [47]:
# keep only those with am label
am_confirmed = am.loc[(am.label == 'am') | (am.label == 'am ')]
am_confirmed.shape

(392, 11)

In [48]:
nam.label.unique()

array(['nam', nan, 'am', 'neither', 'random', 'doubt', 'delete'],
      dtype=object)

In [49]:
nam.loc[nam.label == 'am'].shape, am_confirmed.shape

((19, 11), (392, 11))

In [50]:
am_confirmed = pd.concat([am_confirmed, nam.loc[nam.label == 'am']])

In [51]:
nam.label.unique()

array(['nam', nan, 'am', 'neither', 'random', 'doubt', 'delete'],
      dtype=object)

In [52]:
# keep only those with am label
nam_confirmed = nam.loc[(nam.label == 'nam') | (nam.label == 'random')]
nam_confirmed.shape

(288, 11)

In [53]:
am.loc[am.label == 'nam'].shape

(18, 11)

In [54]:
nam_confirmed = pd.concat([nam_confirmed, am.loc[am.label == 'nam']])
nam_confirmed.shape

(306, 11)